<a href="https://colab.research.google.com/github/tute3a/BusquedasLaborales/blob/main/Busquedas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import re
from datetime import datetime


In [ ]:

driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

total = []
listalinks = []
listalinks_error = []
Hoy = datetime.today().strftime('%d-%m-%Y')

for pagina in range(500):         #Entra página por página
    if pagina > -1:             # Por si se corta en el medio
        #TOTAL url = "https://www.zonajobs.com.ar/ofertas-de-empleo-argentina-pagina-" + str(pagina) + ".html?recientes=true"
        #QUINCENAL url = "https://www.zonajobs.com.ar/ofertas-de-empleo-publicacion-menor-a-15-dias-pagina-" + str(pagina) + ".html?recientes=true"
        #MENSUAL
        url = "https://www.zonajobs.com.ar/ofertas-de-empleo-publicacion-menor-a-1-mes-pagina-" + str(pagina) + ".html?recientes=true"
        driver.get(url)

        avisos = driver.find_element_by_class_name("aviso-no-sponsor")      #Lista de avisos
        links = avisos.find_elements_by_css_selector("a")

        for link in links:          #Busca el link de todos los avisos
            enlace = link.get_attribute('href')
            if enlace != None:                          # Por si no hay enlace dentro de los links que encuentra
                if enlace.find('/empleos/') > 0 and not enlace in listalinks:
                #Toma sólo los links de empleos
                #Links que no se hayan abierto antes
                    sobrecarga = False   # Para que no guarde los errores
                    driver.execute_script("window.open('');")
                    driver.switch_to.window(driver.window_handles[1])
                    driver.get(enlace)              #En una nueva pestaña abre link por link
                    try:                            #Intenta los comandos conocidos (para evitar que falle)

                        try:                        #Comandos para Zonajobs
                            info = driver.find_element_by_class_name("aviso_specs").text
                            descripcion = driver.find_element_by_class_name("aviso_description").text
                            titulo = driver.find_element_by_class_name('aviso_title').text
                            empresa = driver.find_element_by_class_name('aviso_company').text

                            detalle = re.compile("\n(.*)").findall(info)
                            detalle = ((titulo, empresa, detalle[0], detalle[2].replace("Hoy", Hoy), detalle[4], detalle[6], detalle[8],
                            descripcion.replace("/n", "")))

                        except:                     #Comandos para Bumeran

                            titulo = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[1]/div/h1').text
                            empresa = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[1]/div/a/h2').text
                            fecha = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[1]/li[1]/h2').text
                            ubicacion = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[1]/li[2]/a/h2').text
                            fulltime = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[2]/li[1]/a/h2').text
                            sueldo = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[2]/li[2]/h2').text
                            area = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[2]/li[3]/a/h2').text
                            descripcion = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[2]').text
                            detalle = ((titulo, empresa, ubicacion, fecha.replace("Hoy", Hoy), sueldo, fulltime, area, descripcion))


                    except:                         #Salida si no encuentra ni ZonaJobs ni Bumeran
                        sobrecarga = True
                        # Guardar el link
                        el_link = ((enlace))
                        listalinks_error.append(el_link)  # Agrega el link abierto a la lista de links


                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    el_link = ((enlace))
                    listalinks.append(el_link)      #Agrega el link abierto a la lista de links
                    
                    if not sobrecarga:
                        total.append(detalle)           #Si no dio error lo guarda

        print(pagina)
        df = pd.DataFrame(total, columns=['Trabajo', 'Empresa', 'Ubicación', 'Fecha', 'Salario', 'Jornada laboral', 'Área','Descripción'])
        print(df)       #Guarda una vez por página
        df.to_csv(r'ZonaJobsB.csv', index=False, header=True)
        df_error = pd.DataFrame(listalinks_error, columns=['Links con errores'])
        df_error.to_csv(r'ZJErroresB.csv',
                  index=False, header=True)

df = pd.DataFrame(total,columns=['Trabajo','Empresa','Ubicación','Fecha','Salario','Jornada laboral','Área','Descripción'])
print(df)
driver.close()
df.to_csv (r'ZonaJobsB.csv', index = False, header=True)

#Links con errores
df_error = pd.DataFrame(listalinks_error, columns=['Links con errores'])
df_error.to_csv(r'ZJErroresB.csv',
          index=False, header=True)

print("___________ FIN ___________")

## Primer control de los errores

In [ ]:
df = pd.read_csv (r'ZJErrores.csv')
print(len(df))
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
listalinks_error2 = []
total = []
i = 1

for enlace in df['Links con errores']:
                #Toma sólo los links de empleos
                #Links que no se hayan abierto antes
                    print(str(i) + " " + str(round(i/len(df['Links con errores']),2)) + "% " + enlace) # Para ver el número de enlace
                    i = i + 1
                    sobrecarga = False   # Para que no guarde los errores
                    driver.execute_script("window.open('');")
                    driver.switch_to.window(driver.window_handles[1])
                    driver.get(enlace)              #En una nueva pestaña abre link por link
                    try:                            #Intenta los comandos conocidos (para evitar que falle)

                        try:                        #Comandos para Zonajobs
                            info = driver.find_element_by_class_name("aviso_specs").text
                            descripcion = driver.find_element_by_class_name("aviso_description").text
                            titulo = driver.find_element_by_class_name('aviso_title').text
                            empresa = driver.find_element_by_class_name('aviso_company').text

                            detalle = re.compile("\n(.*)").findall(info)
                            detalle = ((titulo, empresa, detalle[0], detalle[2].replace("Hoy", Hoy), detalle[4], detalle[6], detalle[8],
                            descripcion.replace("/n", "")))

                        except:                     #Comandos para Bumeran

                            titulo = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[1]/div/h1').text
                            empresa = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[1]/div/a/h2').text
                            fecha = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[1]/li[1]/h2').text
                            ubicacion = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[1]/li[2]/a/h2').text
                            fulltime = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[2]/li[1]/a/h2').text
                            sueldo = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[2]/li[2]/h2').text
                            area = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[2]/li[3]/a/h2').text
                            descripcion = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[2]').text
                            detalle = ((titulo, empresa, ubicacion, fecha, sueldo, fulltime, area, descripcion))
                            print(detalle)

                    except:                         #Salida si no encuentra ni ZonaJobs ni Bumeran
                        sobrecarga = True
                        # Guardar el link
                        el_link = ((enlace))
                        listalinks_error2.append(el_link)  # Agrega el link abierto a la segunda lista de links
                        # Revisar si podemos guardar algo de info


                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                    if not sobrecarga:
                        total.append(detalle)           #Si no dio error lo guarda
                    print("Total guardados: " + str(len(total)))

df = pd.DataFrame(total,columns=['Trabajo','Empresa','Ubicación','Fecha','Salario','Jornada laboral','Área','Descripción'])
print(df)
driver.close()
df.to_csv (r'ZonaJobs v2.csv', index = False, header=True)

#Links con errores
df_error = pd.DataFrame(listalinks_error2, columns=['Links con errores'])
df_error.to_csv(r'ZJErrores v2.csv',
          index=False, header=True)

print("___________ FIN ___________")

## Segundo control de los errores

In [ ]:
from time import sleep #Agregamos 6 (3+3) segundos de delay

df = pd.read_csv (r'ZJErrores v2.csv')
print(len(df))
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
listalinks_error3 = []
total = []
i = 1

for enlace in df['Links con errores']:
                #Toma sólo los links de empleos
                #Links que no se hayan abierto antes
                    print(str(i) + " " + str(round(i/len(df['Links con errores']),2)) + "% " + enlace) # Para ver el número de enlace
                    i = i + 1
                    sobrecarga = False   # Para que no guarde los errores
                    driver.execute_script("window.open('');")
                    driver.switch_to.window(driver.window_handles[1])
                    driver.get(enlace)              #En una nueva pestaña abre link por link
                    sleep(3) # Para 3 segundos
                    try:                            #Intenta los comandos conocidos (para evitar que falle)

                        try:                        #Comandos para Zonajobs
                            info = driver.find_element_by_class_name("aviso_specs").text
                            descripcion = driver.find_element_by_class_name("aviso_description").text
                            titulo = driver.find_element_by_class_name('aviso_title').text
                            empresa = driver.find_element_by_class_name('aviso_company').text

                            detalle = re.compile("\n(.*)").findall(info)
                            detalle2 = ((titulo, empresa, detalle[0], detalle[2].replace("Hoy", Hoy), detalle[4], detalle[6], detalle[8],
                            descripcion.replace("/n", "")))

                        except:                     #Comandos para Bumeran

                            titulo = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[1]/div/h1').text
                            empresa = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[1]/div/a/h2').text
                            fecha = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[1]/li[1]/h2').text
                            ubicacion = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[1]/li[2]/a/h2').text
                            fulltime = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[2]/li[1]/a/h2').text
                            sueldo = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[2]/li[2]/h2').text
                            area = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/ul[2]/li[3]/a/h2').text
                            descripcion = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div/div/div/div[2]').text
                            detalle2 = ((titulo, empresa, ubicacion, fecha, sueldo, fulltime, area, descripcion))
                            print(detalle2)

                    except:                         #Salida si no encuentra ni ZonaJobs ni Bumeran
                        sobrecarga = True
                        # Guardar el link
                        el_link = ((enlace))
                        listalinks_error3.append(el_link)  # Agrega el link abierto a la segunda lista de links
                        # Revisar si podemos guardar algo de info


                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                    if not sobrecarga:
                        total.append(detalle2)           #Si no dio error lo guarda
                    print("Total guardados: " + str(len(total)))
                    sleep(3) # Para 3 segundos

df = pd.DataFrame(total,columns=['Trabajo','Empresa','Ubicación','Fecha','Salario','Jornada laboral','Área','Descripción'])
print(df)
driver.close()
df.to_csv (r'ZonaJobs v3.csv', index = False, header=True)

#Links con errores
df_error = pd.DataFrame(listalinks_error3, columns=['Links con errores'])
df_error.to_csv(r'ZJErrores v3.csv',
          index=False, header=True)

print("___________ FIN ___________")